In [10]:
import cv2
import numpy as np

In [42]:
# Vid capture
cap = cv2.VideoCapture('/Users/raffaelenardi/Documents/GITHUB/State of the Art Technology/proto_test1.MOV')
whT = 320
confThreshold = 0.5
nms_Throshold = 0.3

classesFile = 'coco.names'
classNames = [] #could just manualy add. maybe faster?

with open(classesFile, 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')

# print(classNames)
# print(len(classNames))

#Architecture and wights
modelConfiguration = 'yolov3.cfg'
modelWeights = 'yolov3.weights'

#Setup of OpenCV as Output
net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

#Find OBJ function
def findObjects(outputs,img):
    hT, wT, cT = img.shape
    bbox = []
    classIds = []
    confs = []
    
    for output in outputs:
        for det in output:
            scores = det[5:] # Removes first 5 elements
            classId = np.argmax(scores) # Finds the index for max conf. score
            confidence = scores[classId] # Saves actual value for the max conf. score
            if confidence > confThreshold: #good detection
                w,h = int(det[2]*wT), int(det[3]*hT) #extracts the w/h values from detection
                x, y = int((det[0]*wT)-w/2), int((det[1]*hT)-h/2) #values in det are cx and cy so they need manipulation
                bbox.append([x,y,w,h]) #appending bboxs
                classIds.append(classId) #append classids
                confs.append(float(confidence)) #append conf. values

    indicies = cv2.dnn.NMSBoxes(bbox,confs,confThreshold,nms_Throshold) #This helps address multiple detections and keeps the best one only

    for i in indicies:
        i = i[0]
        box = bbox[i]
        x,y,w,h = box[0], box[1], box[2], box[3]
        cv2.rectangle(img,(x,y),(x+w,y+h), (255,0,0), 2) #creates bbox in img
        cv2.putText(img,f'{classNames[classIds[i]].upper()} {int(confs[i]*100)}%',
                   (x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2)
                

In [43]:
while True:
    success, img = cap.read()
    
    #Blob is format network understands
    blob = cv2.dnn.blobFromImage(img,1/255,(whT,whT),[0,0,0],1,crop=False)
    net.setInput(blob)
    
    # extracts layer names
    layerNames = net.getLayerNames()
    outputNames = [layerNames[i[0]-1] for i in net.getUnconnectedOutLayers()]
    
    outputs = net.forward(outputNames)
    
    findObjects(outputs,img) #calls function that adds boxes and conf. values
    
    cv2.imshow('img', img)
    cv2.waitKey(1)

KeyboardInterrupt: 